
# NumPy 레퍼런스 문서 / 배열 객체 / 반복

https://numpy.org/doc/stable/reference/arrays.nditer.html

2025/3/13

In [32]:
import numpy as np

a = np.arange(6).reshape(2,3)
for x in np.nditer(a):
    print(x, end=' ')


0 1 2 3 4 5 

- 이 반복에서 주의해야 할 중요한 점은 표준 C 또는 Fortran 순서를 사용하는 대신 배열의 메모리 레이아웃과 일치하도록 순서가 선택된다는 것입니다. 
- 이는 액세스 효율성을 위해 수행되며, 기본적으로 특정 순서를 신경 쓰지 않고 각 요소를 방문하고자 하는 아이디어를 반영합니다. 
- 이 배열의 전치행렬을 이터레이팅 해 보면, order C 로 전치한 후 복사하는 것과 비교 됩니다.

In [42]:
a = np.arange(6).reshape(2,3)
print(a.T)
# T 라는 속성은 '배열의 transposed view' 라고 부른다.
# view 라는 말로부터, 실제로 메모리 복제가 되는 것은 아닌 것 같음.
# 동일한 메모리 데이터를 다른 관점으로 해석하는 것.

for x in np.nditer(a.T):
    print(x, end=' ')

print('')

for x in np.nditer(a.T.copy(order='C')):
    print(x, end=' ')

[[0 3]
 [1 4]
 [2 5]]
0 1 2 3 4 5 
0 3 1 4 2 5 

**numpy.copy(.., order: str = 'K', ..)**:
- order 인자
  - 복사본의 메모리 레이아웃을 제어합니다. 
  - 'C'는 C 언어의 순서를, 'F'는 F 순서를, 'A'는 a가 포트란에 연속된 경우 'F'를, 그렇지 않으면 'C'를 의미합니다. 'K'는 a의 레이아웃과 최대한 가깝게 일치함을 의미합니다.
  - (이 함수와 ndarray.copy는 매우 유사하지만 order= 인수의 기본값이 다릅니다).

In [34]:
a = np.arange(6).reshape(2,3)
print(a)
print(np.copy(a, order='C'))
print(np.copy(a, order='F'))
print(np.copy(a, order='A'))
print(np.copy(a, order='K'))

[[0 1 2]
 [3 4 5]]
[[0 1 2]
 [3 4 5]]
[[0 1 2]
 [3 4 5]]
[[0 1 2]
 [3 4 5]]
[[0 1 2]
 [3 4 5]]


## 반복 순서 제어

## 배열값 수정

op_flags 에 'readwrite' 지정.

- with 블럭을 사용하는 방법.
- close() 를 명시적으로 호출하는 방법.

In [35]:
a = np.arange(6).reshape(2,3)
print(a)

with np.nditer(a, op_flags=['readwrite']) as it:
   print(type(it))
   for x in it:
       #print(type(x))
       print(x)
       x[...] = 2 * x # x[...]는 무엇?

print(a)

[[0 1 2]
 [3 4 5]]
<class 'numpy.nditer'>
0
1
2
3
4
5
[[ 0  2  4]
 [ 6  8 10]]


## 외부 루프 사용

In [36]:
a = np.arange(6).reshape(2,3)
for x in np.nditer(a, flags=['external_loop']):
    print(x, end=' ')
print()
for x in np.nditer(a, flags=['external_loop'], order='F'):
    print(x, end=' ')

[0 1 2 3 4 5] 
[0 3] [1 4] [2 5] 

???????????????????????
이해가 안되는데??

## 인덱스 또는 다중 인덱스 추적 

반복하는 동안 계산에서 현재 요소의 인덱스를 사용하고 싶을 수 있습니다. <br>
인덱스는 반복자 객체 자체에 의해 추적되며, 요청된 내용에 따라 index 또는 multi_index 속성을 통해 액세스할 수 있습니다.

In [37]:
import numpy as np

a = np.arange(6).reshape(2,3) * 3
print(a)

it = np.nditer(a, flags=['c_index'])
for x in it:
    print("[%d]:%d " % (it.index, x), end=' ')
print()
it = np.nditer(a, flags=['f_index'])
for x in it:
    print("[%d]:%d " % (it.index, x), end=' ')

# c_index, f_index 의 차이는 인덱스에만 있고, 순회의 순서는 같다.

[[ 0  3  6]
 [ 9 12 15]]
[0]:0  [1]:3  [2]:6  [3]:9  [4]:12  [5]:15  
[0]:0  [2]:3  [4]:6  [1]:9  [3]:12  [5]:15  

In [38]:
a = np.arange(6).reshape(2,3) * 3

it = np.nditer(a, flags=['multi_index'])
for x in it:
    print("%s:%d " % (it.multi_index, x), end=' ')

(0, 0):0  (0, 1):3  (0, 2):6  (1, 0):9  (1, 1):12  (1, 2):15  

In [39]:
a = np.arange(6).reshape(2,3) * 3
print(a)
with np.nditer(a, flags=['multi_index'], op_flags=['writeonly']) as it:
    for x in it:
        x[...] = it.multi_index[1] - it.multi_index[0]
print(a)

# 대체 이 index 값으로 뭘 더 할 수 있는건가 ?????????????

[[ 0  3  6]
 [ 9 12 15]]
[[ 0  1  2]
 [-1  0  1]]


## 대체 루핑 및 요소 액세스 

- 반복하는 동안 속성에 더 쉽게 액세스할 수 있도록 반복자 객체 자체에서 명시적으로 작동하는 대체 반복 구문이 있습니다. 
- 이 반복 구문을 사용하면 현재 값은 이터레이터에 인덱싱하여 액세스할 수 있습니다. 
- 추적된 인덱스와 같은 다른 속성은 이전과 동일하게 유지됩니다.

In [40]:
import numpy as np

a = np.arange(6).reshape(2,3)
print(a)

it = np.nditer(a, flags=['multi_index'])
while not it.finished:
    print("%s:%d " % (it.multi_index, it[0]), end=' ')
    is_not_finished = it.iternext()
print()

with np.nditer(a, flags=['multi_index'], op_flags=['writeonly']) as it:
    while not it.finished:
        it[0] = it.multi_index[1] - it.multi_index[0]
        is_not_finished = it.iternext()
print(a)

[[0 1 2]
 [3 4 5]]
(0, 0):0  (0, 1):1  (0, 2):2  (1, 0):3  (1, 1):4  (1, 2):5  
[[ 0  1  2]
 [-1  0  1]]



## 배열 요소 버퍼링

## 특정 데이터 유형으로 반복